In [1]:
import pandas as pd
import numpy as np

### The link for the below dataset is "https://drive.google.com/open?id=16LpmQZ-6tx_lxFafuK6VOE7ZkA5XYEx8"

In [4]:
harvey_2014 = pd.read_csv('311-Public-Data-Extract-2014-clean.txt',delimiter='|',infer_datetime_format=True)#,parse_dates=)

In [5]:
harvey_2014.head()

,CASE NUMBER,SR LOCATION,COUNTY,DISTRICT,NEIGHBORHOOD,TAX ID,TRASH QUAD,RECYCLE QUAD,TRASH DAY,HEAVY TRASH DAY,...,SR CREATE DATE,DUE DATE,DATE CLOSED,OVERDUE,Title,x,y,LATITUDE,LONGITUDE,Channel Type
0,0-101001233975,"13803 MILL GARDEN, HOUSTON TX 77059",HARRIS,E,CLEAR LAKE,1172600070011,NaN,NaN,NaN,NaN,...,2014-01-01 00:13:34,2014-09-10 08:30:00,2014-02-28 13:00:18,-193.81,"Nuisance on Property - Trash, Substances, Weed...",3.195276e+06,1.378570e+07,NaN,NaN,Voice In
1,24705-101001233976,"1314 WAVERLY, HOUSTON TX 77008",HARRIS,C,GREATER HEIGHTS,0201720000039,NW,SW,THURSDAY,3rd Wednesday,...,2014-01-01 00:21:16,2014-01-04 00:21:16,2014-01-01 05:59:44,-2.76,Other - 24705,3.108691e+06,1.385409e+07,NaN,NaN,WAP
2,101001233977,"4107 CAVALCADE, HOUSTON TX 77026",HARRIS,B,KASHMERE GARDENS,0660030000026,NE,NW,FRIDAY,1st Thursday,...,2014-01-01 00:23:46,2014-01-08 00:23:46,2014-01-03 16:44:47,-4.32,New Resident Trash Service (First Container)-1...,3.132222e+06,1.385689e+07,29.80056661,-95.33012713,Voice In
3,101001233978,"208 ELEANOR, HOUSTON TX 77009",HARRIS,H,NEAR NORTHSIDE,0660640400022,NE,NW,FRIDAY,1st Friday,...,2014-01-01 00:36:52,2014-01-08 00:36:52,2014-01-03 16:49:37,-4.32,New Resident Trash Service (First Container)-1...,3.119800e+06,1.386069e+07,NaN,NaN,Voice In
4,101001233979,"208 ELEANOR, HOUSTON TX 77009",HARRIS,H,NEAR NORTHSIDE,0660640400022,NE,NW,FRIDAY,1st Friday,...,2014-01-01 00:37:31,2014-01-23 12:40:00,2014-01-09 12:42:18,-14.00,Request a New Recycling Container-101001233979,3.119800e+06,1.386069e+07,NaN,NaN,Voice In


In [6]:
harvey_2014.columns

Index(['CASE NUMBER', 'SR LOCATION', 'COUNTY', 'DISTRICT', 'NEIGHBORHOOD',
       'TAX ID', 'TRASH QUAD', 'RECYCLE QUAD', 'TRASH DAY', 'HEAVY TRASH DAY',
       'RECYCLE DAY', 'KEY MAP', 'MANAGEMENT DISTRICT', 'DEPARTMENT',
       'DIVISION', 'SR TYPE', 'QUEUE', 'SLA', 'STATUS', 'SR CREATE DATE',
       'DUE DATE', 'DATE CLOSED', 'OVERDUE', 'Title', 'x', 'y', 'LATITUDE',
       'LONGITUDE', 'Channel Type'],
      dtype='object')

In [7]:
harvey_2014.rename({'CASE NUMBER':'Case_Number',
       'SR LOCATION':'Request_Location','COUNTY':'County', 'DISTRICT':'District',
       'NEIGHBORHOOD':'Neighborhood', 'TAX ID':'Tax_id',
       'TRASH QUAD':'Trash_Quad', 'RECYCLE QUAD':'Recycle_Quad', 'TRASH DAY':'Trash_Day',
       'HEAVY TRASH DAY':'Heavy_Trash_Day', 'RECYCLE DAY':'Recycle_Day', 'KEY MAP':'Key_Map',
       'MANAGEMENT DISTRICT':'Management_District','DEPARTMENT':'Department',
       'DIVISION':'Division', 'SR TYPE':'SR_Type','QUEUE':'Queue', 
        'SLA':'Service_Level_Agreement_Days','STATUS':'Status', 
               'SR CREATE DATE':'Service_Request_Creation_Date', 'DUE DATE':'Due_Date',
       'DATE CLOSED':'Date_Closed', 'OVERDUE':'Overdue',
       'Title':'Request_Title',
       'x':'X_Coords_ESRI', 'y':'Y_Coords_ESRI', 'LATITUDE':'Latitude',
       'LONGITUDE':'Longitude', 'Channel Type':'Channel_Type', 'Unnamed: 29':'Unnamed_29','month':'Month'},axis=1,inplace=True)

In [8]:
harvey_2014.head()

,Case_Number,Request_Location,County,District,Neighborhood,Tax_id,Trash_Quad,Recycle_Quad,Trash_Day,Heavy_Trash_Day,...,Service_Request_Creation_Date,Due_Date,Date_Closed,Overdue,Request_Title,X_Coords_ESRI,Y_Coords_ESRI,Latitude,Longitude,Channel_Type
0,0-101001233975,"13803 MILL GARDEN, HOUSTON TX 77059",HARRIS,E,CLEAR LAKE,1172600070011,NaN,NaN,NaN,NaN,...,2014-01-01 00:13:34,2014-09-10 08:30:00,2014-02-28 13:00:18,-193.81,"Nuisance on Property - Trash, Substances, Weed...",3.195276e+06,1.378570e+07,NaN,NaN,Voice In
1,24705-101001233976,"1314 WAVERLY, HOUSTON TX 77008",HARRIS,C,GREATER HEIGHTS,0201720000039,NW,SW,THURSDAY,3rd Wednesday,...,2014-01-01 00:21:16,2014-01-04 00:21:16,2014-01-01 05:59:44,-2.76,Other - 24705,3.108691e+06,1.385409e+07,NaN,NaN,WAP
2,101001233977,"4107 CAVALCADE, HOUSTON TX 77026",HARRIS,B,KASHMERE GARDENS,0660030000026,NE,NW,FRIDAY,1st Thursday,...,2014-01-01 00:23:46,2014-01-08 00:23:46,2014-01-03 16:44:47,-4.32,New Resident Trash Service (First Container)-1...,3.132222e+06,1.385689e+07,29.80056661,-95.33012713,Voice In
3,101001233978,"208 ELEANOR, HOUSTON TX 77009",HARRIS,H,NEAR NORTHSIDE,0660640400022,NE,NW,FRIDAY,1st Friday,...,2014-01-01 00:36:52,2014-01-08 00:36:52,2014-01-03 16:49:37,-4.32,New Resident Trash Service (First Container)-1...,3.119800e+06,1.386069e+07,NaN,NaN,Voice In
4,101001233979,"208 ELEANOR, HOUSTON TX 77009",HARRIS,H,NEAR NORTHSIDE,0660640400022,NE,NW,FRIDAY,1st Friday,...,2014-01-01 00:37:31,2014-01-23 12:40:00,2014-01-09 12:42:18,-14.00,Request a New Recycling Container-101001233979,3.119800e+06,1.386069e+07,NaN,NaN,Voice In


In [12]:
harvey_2014['Service_Request_Creation_Date'] = harvey_2014.Service_Request_Creation_Date.dt.date

In [13]:
harvey_2014.head()

,Case_Number,Request_Location,County,District,Neighborhood,Tax_id,Trash_Quad,Recycle_Quad,Trash_Day,Heavy_Trash_Day,...,Service_Request_Creation_Date,Due_Date,Date_Closed,Overdue,Request_Title,X_Coords_ESRI,Y_Coords_ESRI,Latitude,Longitude,Channel_Type
0,0-101001233975,"13803 MILL GARDEN, HOUSTON TX 77059",HARRIS,E,CLEAR LAKE,1172600070011,NaN,NaN,NaN,NaN,...,2014-01-01,2014-09-10 08:30:00,2014-02-28 13:00:18,-193.81,"Nuisance on Property - Trash, Substances, Weed...",3.195276e+06,1.378570e+07,NaN,NaN,Voice In
1,24705-101001233976,"1314 WAVERLY, HOUSTON TX 77008",HARRIS,C,GREATER HEIGHTS,0201720000039,NW,SW,THURSDAY,3rd Wednesday,...,2014-01-01,2014-01-04 00:21:16,2014-01-01 05:59:44,-2.76,Other - 24705,3.108691e+06,1.385409e+07,NaN,NaN,WAP
2,101001233977,"4107 CAVALCADE, HOUSTON TX 77026",HARRIS,B,KASHMERE GARDENS,0660030000026,NE,NW,FRIDAY,1st Thursday,...,2014-01-01,2014-01-08 00:23:46,2014-01-03 16:44:47,-4.32,New Resident Trash Service (First Container)-1...,3.132222e+06,1.385689e+07,29.80056661,-95.33012713,Voice In
3,101001233978,"208 ELEANOR, HOUSTON TX 77009",HARRIS,H,NEAR NORTHSIDE,0660640400022,NE,NW,FRIDAY,1st Friday,...,2014-01-01,2014-01-08 00:36:52,2014-01-03 16:49:37,-4.32,New Resident Trash Service (First Container)-1...,3.119800e+06,1.386069e+07,NaN,NaN,Voice In
4,101001233979,"208 ELEANOR, HOUSTON TX 77009",HARRIS,H,NEAR NORTHSIDE,0660640400022,NE,NW,FRIDAY,1st Friday,...,2014-01-01,2014-01-23 12:40:00,2014-01-09 12:42:18,-14.00,Request a New Recycling Container-101001233979,3.119800e+06,1.386069e+07,NaN,NaN,Voice In


In [19]:
top_20 = ['Storm Debris Collection',
       'Missed Garbage Pickup',
       'Container Problem',
       'Traffic Signal Maintenance',
       'Water Leak',
       'Flooding',
       'Sewer Wastewater',
       'Street Hazard',
       'Nuisance On Property',
       'Drainage',
       'Water Service',
       'Traffic Signs',
       'Crisis Cleanup',
       'Unclassified 311 Web Request',
       'Recycling Cart Repair or Replace',
       'Parking Violation',
       'Traffic Signals',
       'Dead Animal Collection',
       'SWM Escalation',
       'New Resident Container']

In [18]:
# Top 20 SR_Type requests by number of requests
harvey_2014[['Case_Number', 'SR_Type']].groupby('SR_Type').count().sort_values('Case_Number', ascending=False).head(20).reset_index()

,SR_Type,Case_Number
0,Nuisance On Property,28130
1,Container Problem,20155
2,Traffic Signal Maintenance,15277
3,Missed Garbage Pickup,12729
4,Dead Animal Collection,9514
5,Heavy Trash Violation,9042
6,Meter Leak,8185
7,Street Assessment,8105
8,New Resident Container,6963
9,Sewer In Residence or Business,6939


In [ ]:
harvey_date_SR_count = harvey.groupby(['just_date', 'SR_Type']).count().reset_index()

In [32]:
harvey_2014.loc[harvey_2014['SR_Type']\
                .isin(top_20)][['SR_Type','Overdue']].groupby('SR_Type').mean()

,Overdue
SR_Type,
Container Problem,-4.049158
Dead Animal Collection,-2.585769
Drainage,-48.705036
Flooding,-2.464967
Missed Garbage Pickup,-2.683466
New Resident Container,-2.391257
Nuisance On Property,-81.242603
Parking Violation,-8.336415
Recycling Cart Repair or Replace,-5.750439
